<a href="https://colab.research.google.com/github/Chetnajainmj/Machine-Learning/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Function to convert grayscale (28x28) → RGB (224x224)
def preprocess_mnist(image, label):
    image = tf.expand_dims(image, -1)                       # shape: (28, 28, 1)
    image = tf.image.grayscale_to_rgb(image)                # shape: (28, 28, 3)
    image = tf.image.resize(image, [224, 224])              # shape: (224, 224, 3)
    image = preprocess_input(image)                         # VGG preprocessing
    return image, label

In [4]:
# Create tf.data.Dataset objects (no memory overflow)
batch_size = 64

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
          .map(preprocess_mnist) \
          .shuffle(1000) \
          .batch(batch_size) \
          .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
         .map(preprocess_mnist) \
         .batch(batch_size) \
         .prefetch(tf.data.AUTOTUNE)

In [5]:
# Load VGG16 base model without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False  # Freeze the base

# Add classification head
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_ds, epochs=3, validation_data=test_ds)

Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


938/938 ━━━━━━━━━━━━━━━━━━━━ 369s 368ms/step - accuracy: 0.9458 - loss: 0.9462 - val_accuracy: 0.9841 - val_loss: 0.0691
Epoch 2/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 360s 382ms/step - accuracy: 0.9897 - loss: 0.0422 - val_accuracy: 0.9869 - val_loss: 0.0930
Epoch 3/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 361s 383ms/step - accuracy: 0.9918 - loss: 0.0468 - val_accuracy: 0.9864 - val_loss: 0.0876


In [7]:
# Evaluate on test set
loss, accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

157/157 ━━━━━━━━━━━━━━━━━━━━ 47s 297ms/step - accuracy: 0.9834 - loss: 0.1034
Test Accuracy: 98.64%
